In [1]:
#!pip install keras
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Add, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import glorot_uniform
from keras import optimizers
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
num_classes  = 10
batch_size   = 128
epochs       = 50
dropout      = 0.5
weight_decay = 0.0001

In [3]:
from keras import backend as K
if('tensorflow' == K.backend()):
    import tensorflow as tf
    from keras.backend.tensorflow_backend import set_session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255

In [5]:
def res_block(X, f, filters, stage, block, increase = False, s = 2):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters   # Retrieving filters
    X_shortcut = X         # Saving input value
    stride = (1,1)
    
    if increase:
        stride = (s,s)
        
        #Shortcut Path
        X_shortcut = Conv2D(F3, (1,1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
        X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = stride, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters  = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

In [6]:
def ResNet50(input_shape, classes = num_classes):
    
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0), padding = 'same')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = res_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1, increase = True)
    X = res_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = res_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (≈4 lines)
    X = res_block(X, f = 3, filters = [128,128,512], stage = 3, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'b')
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'c')
    X = res_block(X, 3, [128,128,512], stage = 3, block = 'd')
    
    # Stage 4 (≈6 lines)
    X = res_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'b')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'c')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'd')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'e')
    X = res_block(X, 3, [256, 256, 1024], stage = 4, block = 'f')
    
    # Stage 5 (≈3 lines)
    X = res_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = 'a', s = 2, increase = True)
    X = res_block(X, 3, [512, 512, 2048], stage = 5, block = 'b')
    X = res_block(X, 3, [512, 512, 2048], stage = 5, block = 'c')
    
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    resnet = Model(inputs = X_input, outputs = X, name='ResNet50')

    return resnet

In [7]:
input_shape = x_train.shape[1:]
resnet = ResNet50(input_shape, classes = num_classes)
resnet.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 19, 19, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 19, 19, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [8]:
resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
resnet.fit(x_train, y_train, epochs = 10, batch_size = 128)

Epoch 1/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.5643 - accuracy: 0.8006
Epoch 2/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.4910 - accuracy: 0.8272
Epoch 3/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.4481 - accuracy: 0.8399
Epoch 4/10
50000/50000 [==============================] - 76s 2ms/step - loss: 0.4301 - accuracy: 0.8496
Epoch 5/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.3663 - accuracy: 0.8713
Epoch 6/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.2776 - accuracy: 0.9030
Epoch 7/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.2421 - accuracy: 0.9136
Epoch 8/10
50000/50000 [==============================] - 75s 2ms/step - loss: 0.3155 - accuracy: 0.8960
Epoch 9/10
50000/50000 [==============================] - 76s 2ms/step - loss: 0.3774 - accuracy: 0.8698
Epoch 10/10
50000/50000 [==============================

In [12]:
scores = resnet.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 1.6892033784866334
Test accuracy: 0.6255999803543091


In [13]:
resnet.save('resnet50.h5')
resnet.save_weights('resnet50_weights.h5')